## HotChili Analytics trading notebook template
#### Configure by setting ALGO_NAME in cell below.
#### Run various options (backtest, ingest, live) by uncommenting one cell 

In [1]:
%matplotlib inline
%load_ext zipline

# %reload_ext zipline # Uncomment and use this when already loaded zipline extension for magic cell usage.

/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/odo/backends/pandas.py:94: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))


In [2]:
import pandas as pd

# Options you can uncomment and set:

# pd.set_option("max_colwidth", 300)
# pd.set_option("display.max_rows", 300)
# pd.set_option("display.max_columns", 50)
# pd.set_option('precision', 2)
# pd.options.display.float_format = '{:20,.2f}'.format

In [3]:
import os

hca_root_path = os.environ['HCA_ROOT']
print(f"hca_root_path = {hca_root_path}")

hca_root_path = /home/hca-ws2004/hca


# Construct algorithm strategy path names

Assumptions:

- the strategy is in a directory with the same name as the strategy in `ALGO_NAME` below
- the strategy is located in the hca-resources directory, which is located relative to `hca_root_path`, found above

In [4]:
ALGO_NAME = "HCA_In_and_Out" # <--- Supply name here

Other variables are derived from `ALGO_NAME`:

In [5]:
HCA_RESOURCES_PATH = hca_root_path + "/hca-resources/" 
ALGO_PATH          = HCA_RESOURCES_PATH + ALGO_NAME + "/" 
ALGO_BT            = ALGO_PATH + ALGO_NAME + ".py"
ALGO_BT_OUT        = ALGO_PATH + ALGO_NAME + ".pkl"
ALGO_LIVE          = ALGO_PATH + ALGO_NAME + "_Live" + ".py"

print(f"""
ALGO_NAME          = {ALGO_NAME}
HCA_RESOURCES_PATH = {HCA_RESOURCES_PATH}
ALGO_PATH          = {ALGO_PATH}
ALGO_BT            = {ALGO_BT}
ALGO_BT_OUT        = {ALGO_BT_OUT}
ALGO_LIVE          = {ALGO_LIVE}

Contents of algo directory:
""")

!ls $ALGO_PATH


ALGO_NAME          = HCA_In_and_Out
HCA_RESOURCES_PATH = /home/hca-ws2004/hca/hca-resources/
ALGO_PATH          = /home/hca-ws2004/hca/hca-resources/HCA_In_and_Out/
ALGO_BT            = /home/hca-ws2004/hca/hca-resources/HCA_In_and_Out/HCA_In_and_Out.py
ALGO_BT_OUT        = /home/hca-ws2004/hca/hca-resources/HCA_In_and_Out/HCA_In_and_Out.pkl
ALGO_LIVE          = /home/hca-ws2004/hca/hca-resources/HCA_In_and_Out/HCA_In_and_Out_Live.py

Contents of algo directory:

HCA_In_and_Out.ipynb	HCA_In_and_Out_Q.py
HCA_In_and_Out_Live.py	HCA_In_and_Out_tearsheet.html
HCA_In_and_Out.pkl	HCA_In_and_Out_tearsheet.ipynb
HCA_In_and_Out.py	Q_In_and_Out_orig.py


## Zipline backtest: 

- Method: Jupyter magic cell (%%) 
- Execution of zipline code in cell containing command line command
- Uncomment first line and hit (shift-enter) inside the cell to run simulation backtest

In [6]:
#%%zipline --start=2018-1-1 --end=2020-8-10 -b sharadar-eqfd -o $ALGO_BT_OUT


## Display your current bundles

In [7]:
#!zipline bundles # Finds all bundles

## Ingest Sharadar funds assets for today

In [8]:
# Ingest Sharadar funds assets for today, if needed.

# Only need to ingest Funds for this algo, and this takes less processing time and system memory than ingesting
# all of Sharadar Equities plus Funds bundle (sharadar-eqfd)
# !zipline ingest -b sharadar-funds

## Zipline backtest, alternative method

- Method: command line
- Execution of zipline code, located in a file, using below command line execution with magic (`!`) invocation
- This line can also be run in a terminal by copying everything past the `!` and pasting (shift-insert) it into the target terminal

In [9]:
!zipline run -f $ALGO_BT  --start=2018-1-1 --end=2021-04-29 --capital-base 10000 -b sharadar-funds -o $ALGO_BT_OUT

/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/odo/backends/pandas.py:94: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))
extension: hca_root_path = /home/hca-ws2004/hca
extension:TODAY_STR = 2021-05-03
extension:TWO_YR_AGO_STR = 2019-05-03
extension:  start_date=2019-05-03 end_date = 2021-05-03
[2021-05-03 22:43:33.601687] INFO: Loader: Cache at /home/hca-ws2004/zipline-broker/data/SPY_benchmark.csv does not have data from 1990-01-02 00:00:00+00:00 to 2021-05-03 00:00:00+00:00.

[2021-0

## Run Zipline live on IB-TWS via command line

- Method: command line
- Execution of zipline code using below command line execution using magic (`!`) invocation

**Notes:** 
- IB-TWS or IB-Gateway must be running, with `IB_ACCT` and `IB_URI` port being correct to live trade
- Change `I_WANT_TO_RUN_THIS_CODE` to `True` below to run zipline live on IB-TWS/IB-Gateway

In [10]:
TODAY = pd.datetime.today().strftime("%Y-%m-%d")
print("TODAY = {}".format(TODAY))

TODAY = 2021-05-03


In [11]:
ALGO_STATE = ALGO_PATH + "strategy.state" 
ALGO_RTB   = ALGO_PATH + "realtime-bars/"

# Edit the following URI to match your IB account and port info.
IB_ACCT = "DU1568488"
IB_URI = "127.0.0.1:7497:1301"

# Change following to 'True' and run cell (control-enter) to execute live run.
I_WANT_TO_RUN_THIS_CODE = False
#I_WANT_TO_RUN_THIS_CODE = True

if I_WANT_TO_RUN_THIS_CODE:
    
    !zipline run \
        -s $TODAY \
        -f $ALGO_LIVE \
        --bundle hca-symbol \
        --broker ib \
        --broker-uri $IB_URI \
        --broker-acct $IB_ACCT \
        --data-frequency daily \
        --state-file $ALGO_STATE \
        --realtime-bar-target $ALGO_RTB 